In [6]:
!pip install loica

from loica import *
import matplotlib.pyplot as plt
import numpy as np
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Simulate repressilator gene network
def gen_image(genetic_parameters, count):
    N = 3

    ring = GeneticNetwork()

    regs = [Regulator(name=f'Rep{i}', degradation_rate=genetic_parameters[i][0]) for i in range(N)]
    ring.add_regulator(regs)

    reps = [Reporter(name=f'SFP{i}', color='blue', degradation_rate=genetic_parameters[i][0]) for i in range(N)]
    ring.add_reporter(reps)


    nots = [Hill1(input=regs[i], output=regs[(i+1)%N], alpha=genetic_parameters[i][1], K=1, n=2) for i in range(N)]
    ring.add_operator(nots)

    nots = [Hill1(input=regs[i], output=reps[i], alpha=genetic_parameters[i][1], K=1, n=2) for i in range(N)]
    ring.add_operator(nots)


    def growth_rate(t):
        return 1

    def biomass(t):
        return 1

    metab = SimulatedMetabolism("name", biomass, growth_rate)


    regs[0].init_concentration = 5
    reps[0].init_concentration = 5


    col = Colony(ring, 1, 1)
    kymo = col.kymograph(nx=250, t0=0, tmax=48)


    # Figsize controls simulated image resolution with 1, 1 being 72x72
    fig, ax = plt.subplots(figsize=(1, 1))
    # Configure and save plot
    plt.imshow(col.map_kymo(col.norm_kymo(kymo)), aspect='auto')
    ax.set_position([0, 0, 1, 1])
    plt.axis("off")
    plt.savefig("drive/MyDrive/Colab Notebooks/Raw Data 10000x72/image" + str(count) + ".jpg")
    plt.close()

In [9]:
# Number of images produced = (n+1)^2
n = 99
alpha_max = 1000000
alpha_min = 1000
deg_max = 1
deg_min = 1e-6

deg_step = (deg_max - deg_min) / n
alpha_step = (alpha_max - alpha_min) / n

genetic_parameters = []

# Construct genetic parameters array
for i in range(n + 1):
    for j in range(n + 1):
        deg_val = deg_min + (i * deg_step)
        alpha_val = alpha_min + (j * alpha_step)
        genetic_parameters.append([deg_val, alpha_val])
        j += 1
    i += 1

genetic_parameters = np.asarray(genetic_parameters)
np.save("drive/MyDrive/Colab Notebooks/Raw Data 10000x72/genetic_parameters.npy", genetic_parameters)

In [10]:
# Image range to produce, distance between min and max > 250 can cause memory error
min = 0
max = 250

# Simulate kymograph images 
start_time = time.perf_counter()
for i in range(genetic_parameters.shape[0]):
    if min <= i <= max:
        gen_image([[genetic_parameters[i][0], [genetic_parameters[i][1], 0]]]*3, i)
        print(i, genetic_parameters[i][0], genetic_parameters[i][1])
    elif i > max:
        break
end_time = time.perf_counter()
print(f"Execution Time : {end_time - start_time:0.6f}")

0 1e-06 1000.0
1 1e-06 11090.90909090909
2 1e-06 21181.81818181818
3 1e-06 31272.727272727272
4 1e-06 41363.63636363636
5 1e-06 51454.54545454545
6 1e-06 61545.454545454544
7 1e-06 71636.36363636363
8 1e-06 81727.27272727272
9 1e-06 91818.18181818181
10 1e-06 101909.0909090909
11 1e-06 111999.99999999999
12 1e-06 122090.90909090909
13 1e-06 132181.81818181818
14 1e-06 142272.72727272726
15 1e-06 152363.63636363635
16 1e-06 162454.54545454544
17 1e-06 172545.45454545453
18 1e-06 182636.36363636362
19 1e-06 192727.2727272727
20 1e-06 202818.1818181818
21 1e-06 212909.09090909088
22 1e-06 222999.99999999997
23 1e-06 233090.90909090906
24 1e-06 243181.81818181818
25 1e-06 253272.72727272726
26 1e-06 263363.63636363635
27 1e-06 273454.5454545454
28 1e-06 283545.45454545453
29 1e-06 293636.3636363636
30 1e-06 303727.2727272727
31 1e-06 313818.18181818177
32 1e-06 323909.0909090909
33 1e-06 334000.0
34 1e-06 344090.90909090906
35 1e-06 354181.8181818182
36 1e-06 364272.72727272724
37 1e-06 37